# Run the final state algorithm on a ROOT HSDATA file

In [1]:
gROOT->ProcessLine(".x LoadK2.C+");

%%%%%%%%%%%%%%%%%%%%%%%%%    THSParticle
%%%%%%%%%%%%%%%%%%%%%%%%%    EventInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    RunInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    THSWeights
%%%%%%%%%%%%%%%%%%%%%%%%%    DataManager
%%%%%%%%%%%%%%%%%%%%%%%%%    LundReader
&&&&&&&&&&&&&&&&&&&&&&&&&&&& HSKinematics
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Cuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Combitorial
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleIter
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Topology
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FinalState
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FiledTree
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreeData
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TopoActionManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& VarsParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalID
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCutsManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalIDManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS/CLASTrigger
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS12/CLAS12

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.

In [2]:
 K2 fs("Beam","Beam");
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

testing Beam Beam
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
Incusive particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 Topology::Print() : 0
    particles = -10000 -22 10000 10000 
You can have any number of the following particles : 
-22 
The following particle are identified by pdg code : 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [3]:
auto output = new HS::FiledTree("tt","testcuts.root");
fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

**Making your own cuts class** Here I use a new cuts class DeltaTimePCut. You can make your own cuts follwing the example notebook [DeltaTimePCut](MakeDeltaTimePCut.ipynb)

In [4]:
ParticleCutsManager cutsman;
auto beamCut=make_shared<DeltaTimePCut>(1,3); //7ns delta time cut
auto otherCut=make_shared<DeltaTimePCut>(10,3); //10ns cut for all other particles
cutsman.AddParticleCut("Beam",beamCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

Info ParticleCutsManager AddParticleCut set cut for 0
Info ParticleCutsManager AddParticleCut set cut for -22


Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined. 

In [5]:
fs.RegisterPostTopoAction(&cutsman);

**Particle Monitoring** here we are going to make seperate trees for each topology and only keep the detected particle information in each one.

In [6]:
TreePrepManager treeman("/scratch/dglazier/g8/particles/t3");
treeman.SetParticleVars("Beam",{"P","Time"}); //For beam only interesetef in energy and time
treeman.ConfigureTreeParticles(&fs); //propogate through topologies
fs.RegisterPostWorkAction(&treeman); //register post-work i.e. after kinematics is called and all calculations

Adding Beam
    Current number of particles: 0
 Resized vectors to fill: 1, 1
Added 2 (Float_t) / 
Current particles: Beam  
Adding Proton
    Current number of particles: 1
 Resized vectors to fill: 2, 2
Added 6 (Float_t) / 
Current particles: Beam Proton  
Adding Kp
    Current number of particles: 2
 Resized vectors to fill: 3, 3
Added 6 (Float_t) / 
Current particles: Beam Proton Kp  
Adding Km
    Current number of particles: 3
 Resized vectors to fill: 4, 4
Added 6 (Float_t) / 
Current particles: Beam Proton Kp Km  
Setting branches...
Printing variables: 
            BeamP   0 / 0
            BeamTime   0 / 1
            ProtonP   1 / 0
            ProtonTh   1 / 1
            ProtonPhi   1 / 2
            ProtonTime   1 / 3
            ProtonEdep   1 / 4
            ProtonDeltaE   1 / 5
            KpP   2 / 0
            KpTh   2 / 1
            KpPhi   2 / 2
            KpTime   2 / 3
            KpEdep   2 / 4
            KpDeltaE   2 / 5
            KmP   3 / 0
            

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [7]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/w/work1/jlab/hallb/HS_g8/1_9_list/flist_048088.dat.root");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Info in <DataManager::InitChain>:  Will proceess all 1 files in chain


DataManager::InitReader set branch 0x7f2b50533a28 0x7f2b50533a28


Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [8]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;

fs.ProcessData(); //No number give, analyse all events in chain
cout<<"Done "<<dm->GetEntry()<< " events "<<endl;

Number of Events to process 1111760
 ProcessData event 100000
 ProcessData event 200000
 ProcessData event 300000
 ProcessData event 400000
 ProcessData event 500000
 ProcessData event 600000
 ProcessData event 700000
 ProcessData event 800000
 ProcessData event 900000
 ProcessData event 1000000
 ProcessData event 1100000
1111760 1 1
DataManager::NextChainFile() Finished all files 
******TreeCache statistics for tree: HSParticles in file: /w/work1/jlab/hallb/HS_g8/1_9_list/flist_048088.dat.root ******
TreeCache = 50 MBytes
N leaves  = 29
ReadTotal = 263.484 MBytes
ReadUnZip = 717.401 MBytes
ReadCalls = 952
ReadSize  = 276.769 KBytes/read
Readahead = 256 KBytes
Readextra =  0.00 per cent
Real Time =  36.556 seconds
CPU  Time =  36.510 seconds
Disk Time =   0.225 seconds
Disk IO   = 1171.765 MBytes/s
ReadUZRT  =  19.625 MBytes/s
ReadUZCP  =  19.649 MBytes/s
ReadRT    =   7.208 MBytes/s
ReadCP    =   7.217 MBytes/s
Done 1111760 events 


And remember to save the tree!

In [9]:
//delete output;

In [10]:
%jsroot on

In [14]:
//TCanvas c3;
//c3.Divide(2,2);
c3.cd(1);
cout<<" rh "<<treeman.GetPrep(1)<<endl;
treeman.GetPrep(0)->Tree()->Draw("BeamP>>h1(100,0,3)","");
c3.cd(2);
treeman.GetPrep(0)->Tree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,4)","","col1");
c3.cd(3);
treeman.GetPrep(0)->Tree()->Draw("KpP:KpTime>>h3(100,-10,10,100,0,4)","","col1");
c3.cd(4);
treeman.GetPrep(0)->Tree()->Draw("KpP:KpTime>>h4(100,-10,10,100,0,4)","","col1");


 rh 0x7f2b502fe740


In [15]:
c3.Draw();

**When ready delete the treeman so trees as saved to the output directory**

In [13]:
//delete &treeman;